In [2]:
import pandas as pd
import json

# MSOA

In [3]:
# Which MSOAs are in Suffolk?
df = pd.read_csv("MSOA_lookups.csv")
# Bury St Edmunds, Ipswich, Mid Suffolk, Rochford and South Suffolk
#Babergh, East Suffolk, Mid-Suffolk, Ipswich and West Suffolk
suffolk_msoas = df[df.LAD22NM.str.contains("Suffolk", case=False)|df.LAD22NM.str.contains("Babergh", case=False)|df.LAD22NM.str.contains("East Suffolk", case=False)|df.LAD22NM.str.contains("Mid-Suffolk", case=False)|df.LAD22NM.str.contains("Ipswich", case=False)|df.LAD22NM.str.contains("West Suffolk", case=False)]
suffolk_msoas = suffolk_msoas.MSOA21CD

df = pd.read_csv("industry_by_MSOA.csv", skiprows=5)
# Index(['2021 super output area - middle layer', 'mnemonic', 'Total', '%',
#        'A Agriculture, forestry and fishing', '%.1', 'B Mining and quarrying',
#        '%.2', 'C Manufacturing', '%.3',
#        'D Electricity, gas, steam and air conditioning supply', '%.4',
#        'E Water supply; sewerage, waste management and remediation activities',
#        '%.5', 'F Construction', '%.6',
#        'G Wholesale and retail trade; repair of motor vehicles and motor cycles',
#        '%.7', 'H Transport and storage', '%.8',
#        'I Accommodation and food service activities', '%.9',
#        'J Information and communication', '%.10',
#        'K Financial and insurance activities', '%.11',
#        'L Real estate activities', '%.12',
#        'M Professional, scientific and technical activities', '%.13',
#        'N Administrative and support service activities', '%.14',
#        'O Public administration and defence; compulsory social security',
#        '%.15', 'P Education', '%.16',
#        'Q Human health and social work activities', '%.17', 'R, S, T, U Other',
#        '%.18'],

df.columns = [
    "MSOA",
    "id",
    "total_level",
    "total_percent",
    "agriculture",
    "agriculture_percent",
    "mining",
    "mining_percent",
    "manufacturing",
    "manufacturing_percent",
    "electricity",
    "electricity_percent",
    "water_supply",
    "water_supply_percent",
    "construction",
    "construction_percent",
    "wholesale_retail",
    "wholesale_retail_percent",
    "transport",
    "transport_percent",
    "accommodation_food",
    "accommodation_food_percent",
    "information_communication",
    "information_communication_percent",
    "financial_insurance",
    "financial_insurance_percent",
    "real_estate",
    "real_estate_percent",
    "professional_scientific",
    "professional_scientific_percent",
    "administrative_support",
    "administrative_support_percent",
    "public_administration",
    "public_administration_percent",
    "education",
    "education_percent",
    "health_social_work",
    "health_social_work_percent",
    "other",
    "other_percent"
]

# drop the level cols
df = df.drop(columns=[c for c in df.columns if "level" in c or "other" in c])
df = df[df.id.isin(suffolk_msoas)]
df

,MSOA,id,total_percent,agriculture,agriculture_percent,mining,mining_percent,manufacturing,manufacturing_percent,electricity,...,professional_scientific,professional_scientific_percent,administrative_support,administrative_support_percent,public_administration,public_administration_percent,education,education_percent,health_social_work,health_social_work_percent
3869,Babergh 001,E02006227,100.0,125,4.4,1,0.0,199,7.0,15,...,210,7.4,160,5.7,158,5.6,266,9.4,336,11.9
3870,Babergh 002,E02006228,100.0,82,2.8,2,0.1,255,8.6,11,...,197,6.6,189,6.4,131,4.4,263,8.9,400,13.5
3871,Babergh 003,E02006229,100.0,43,1.6,0,0.0,252,9.2,16,...,175,6.4,154,5.6,130,4.8,255,9.3,338,12.4
3872,Babergh 004,E02006230,100.0,95,1.9,0,0.0,351,7.0,30,...,280,5.6,274,5.5,417,8.4,490,9.8,606,12.2
3873,Babergh 005,E02006231,100.0,57,1.2,6,0.1,272,6.0,40,...,268,5.9,218,4.8,331,7.3,393,8.6,554,12.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4020,West Suffolk 018,E02006283,100.0,96,2.6,2,0.1,356,9.6,23,...,310,8.4,211,5.7,265,7.1,321,8.7,483,13.0
4021,West Suffolk 019,E02006284,100.0,24,0.6,2,0.1,548,14.3,15,...,222,5.8,190,5.0,177,4.6,326,8.5,627,16.3
4022,West Suffolk 021,E02006285,100.0,15,0.4,1,0.0,527,13.1,13,...,285,7.1,221,5.5,151,3.7,336,8.3,717,17.8
4023,West Suffolk 020,E02006286,100.0,25,0.4,3,0.1,807,14.5,20,...,286,5.1,285,5.1,231,4.1,382,6.9,917,16.5


In [32]:
with open("Middle_layer_Super_Output_Areas_December_2021_Boundaries_EW_BGC_V3_4916445166053426.geojson") as f:
    geo_data = json.load(f)

geo_data['features'] = [f for f in geo_data['features'] if f['properties']['MSOA21CD'] in suffolk_msoas.values]

# add the data to the geojson
for f in geo_data['features']:
    msoa = f['properties']['MSOA21CD']
    data = df[df.id == msoa]
    f['properties']["data"] = {}
    for k in data.columns:
        if k == "MSOA":
            continue
        if str(data[k].values[0]).isnumeric():
            f['properties'][k] = float(data[k].values[0])
        else:
            f['properties'][k] = (data[k].values[0])

with open("suffolk_msoas.geojson", "w") as f:
    json.dump(geo_data, f)

In [ ]:
geo_data['features'][0]['properties']
{'FID': 5985,
 'MSOA21CD': 'E02006227',
 'MSOA21NM': 'Babergh 001',
 'MSOA21NMW': ' ',
 'BNG_E': 595852,
 'BNG_N': 250780,
 'LAT': 52.12065,
 'LONG': 0.859403,
 'GlobalID': 'b91faf31-cf32-4889-9b28-40a6701f0b5d',
 'data': {'id': 'E02006227',
  'total_percent': 100.0,
  'agriculture': 125.0,
  'agriculture_percent': 4.4,
  'mining': 1.0,
  'mining_percent': 0.0,
  'manufacturing': 199.0,
  'manufacturing_percent': 7.0,
  'electricity': 15.0,
  'electricity_percent': 0.5,
  'water_supply': 10.0,
  'water_supply_percent': 0.4,
  'construction': 348.0,
  'construction_percent': 12.3,
  'wholesale_retail': 387.0,
  'wholesale_retail_percent': 13.7,
  'transport': 85.0,
  'transport_percent': 3.0,
  'accommodation_food': 132.0,
  'accommodation_food_percent': 4.7,
  'information_communication': 88.0,
  'information_communication_percent': 3.1,
  'financial_insurance': 88.0,
  'financial_insurance_percent': 3.1,
  'real_estate': 51.0,
  'real_estate_percent': 1.8,
  'professional_scientific': 210.0,
  'professional_scientific_percent': 7.4,
  'administrative_support': 160.0,
  'administrative_support_percent': 5.7,
  'public_administration': 158.0,
  'public_administration_percent': 5.6,
  'education': 266.0,
  'education_percent': 9.4,
  'health_social_work': 336.0,
  'health_social_work_percent': 11.9}}

{'FID': 5985,
 'MSOA21CD': 'E02006227',
 'MSOA21NM': 'Babergh 001',
 'MSOA21NMW': ' ',
 'BNG_E': 595852,
 'BNG_N': 250780,
 'LAT': 52.12065,
 'LONG': 0.859403,
 'GlobalID': 'b91faf31-cf32-4889-9b28-40a6701f0b5d',
 'data': {'id': 'E02006227',
  'total_percent': 100.0,
  'agriculture': 125.0,
  'agriculture_percent': 4.4,
  'mining': 1.0,
  'mining_percent': 0.0,
  'manufacturing': 199.0,
  'manufacturing_percent': 7.0,
  'electricity': 15.0,
  'electricity_percent': 0.5,
  'water_supply': 10.0,
  'water_supply_percent': 0.4,
  'construction': 348.0,
  'construction_percent': 12.3,
  'wholesale_retail': 387.0,
  'wholesale_retail_percent': 13.7,
  'transport': 85.0,
  'transport_percent': 3.0,
  'accommodation_food': 132.0,
  'accommodation_food_percent': 4.7,
  'information_communication': 88.0,
  'information_communication_percent': 3.1,
  'financial_insurance': 88.0,
  'financial_insurance_percent': 3.1,
  'real_estate': 51.0,
  'real_estate_percent': 1.8,
  'professional_scientific':

In [ ]:
"foo".isnu

In [30]:
temp_df = df[[c for c in df.columns if ("percent" in c or "idz" in c or "MSOAx" in c) and "total" not in c]]
# what's the biggest percentage in the whole dataset?
temp_df.max().max()

25.0

In [6]:
f['properties']["data"]

{}

In [ ]:
df[df.id==]

,MSOA,id,total_percent,agriculture,agriculture_percent,mining,mining_percent,manufacturing,manufacturing_percent,electricity,...,professional_scientific,professional_scientific_percent,administrative_support,administrative_support_percent,public_administration,public_administration_percent,education,education_percent,health_social_work,health_social_work_percent
3869,Babergh 001,E02006227,100.0,125,4.4,1,0.0,199,7.0,15,...,210,7.4,160,5.7,158,5.6,266,9.4,336,11.9
3870,Babergh 002,E02006228,100.0,82,2.8,2,0.1,255,8.6,11,...,197,6.6,189,6.4,131,4.4,263,8.9,400,13.5
3871,Babergh 003,E02006229,100.0,43,1.6,0,0.0,252,9.2,16,...,175,6.4,154,5.6,130,4.8,255,9.3,338,12.4
3872,Babergh 004,E02006230,100.0,95,1.9,0,0.0,351,7.0,30,...,280,5.6,274,5.5,417,8.4,490,9.8,606,12.2
3873,Babergh 005,E02006231,100.0,57,1.2,6,0.1,272,6.0,40,...,268,5.9,218,4.8,331,7.3,393,8.6,554,12.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4020,West Suffolk 018,E02006283,100.0,96,2.6,2,0.1,356,9.6,23,...,310,8.4,211,5.7,265,7.1,321,8.7,483,13.0
4021,West Suffolk 019,E02006284,100.0,24,0.6,2,0.1,548,14.3,15,...,222,5.8,190,5.0,177,4.6,326,8.5,627,16.3
4022,West Suffolk 021,E02006285,100.0,15,0.4,1,0.0,527,13.1,13,...,285,7.1,221,5.5,151,3.7,336,8.3,717,17.8
4023,West Suffolk 020,E02006286,100.0,25,0.4,3,0.1,807,14.5,20,...,286,5.1,285,5.1,231,4.1,382,6.9,917,16.5


In [13]:
msoa

'E02006227'

In [11]:
data

,MSOA,id,total_percent,agriculture,agriculture_percent,mining,mining_percent,manufacturing,manufacturing_percent,electricity,...,professional_scientific,professional_scientific_percent,administrative_support,administrative_support_percent,public_administration,public_administration_percent,education,education_percent,health_social_work,health_social_work_percent


In [10]:
f

{'type': 'Feature',
 'id': 5985,
 'geometry': {'type': 'Polygon',
  'coordinates': [[[0.793078663152347, 52.1756405525967],
    [0.795466910074836, 52.1751479286483],
    [0.796623719533912, 52.1752110013669],
    [0.798401442394665, 52.174373220568],
    [0.800067187445001, 52.1744462430848],
    [0.804722162255388, 52.1736511458848],
    [0.807685597685444, 52.1721502941048],
    [0.810173583220128, 52.171904473725],
    [0.814169635584244, 52.1707325527536],
    [0.813720343124407, 52.1704348073572],
    [0.814386098053686, 52.1689297334512],
    [0.814681649451741, 52.1689791749939],
    [0.815145809737672, 52.1691222119462],
    [0.816600856285102, 52.1682213187542],
    [0.821180400609133, 52.1641018717899],
    [0.825579142860185, 52.1626671084785],
    [0.829770746325175, 52.1627251025357],
    [0.832911429425867, 52.1619803934242],
    [0.838006767209165, 52.1618236907601],
    [0.843326190651277, 52.1612656754321],
    [0.846965295706825, 52.1617625601374],
    [0.85368300500

In [9]:
data

,MSOA,id,total_percent,agriculture,agriculture_percent,mining,mining_percent,manufacturing,manufacturing_percent,electricity,...,professional_scientific,professional_scientific_percent,administrative_support,administrative_support_percent,public_administration,public_administration_percent,education,education_percent,health_social_work,health_social_work_percent


In [5]:
k

'id'

{'FID': 1,
 'MSOA21CD': 'E02000001',
 'MSOA21NM': 'City of London 001',
 'MSOA21NMW': ' ',
 'BNG_E': 532384,
 'BNG_N': 181355,
 'LAT': 51.51562,
 'LONG': -0.09349,
 'GlobalID': '71249043-b176-4306-ba6c-d1a993b1b741'}